# Предобработка данных и их подготовка к анализу и подаче в модель.

Под предобработкой понимаются следующие этапы работы с данными:

* очистка данных от аномальных значений (выбросов);
* работа с пропущенными значениями;
* удаление признаков, которые не несут полезной информации;
* создание новых признаков;
* преобразование признаков и приведение данных к необходимому для анализа и модели формату.

In [58]:
import pandas as pd

## FEATURE ENGINEERING
или __генерацией признаков (фичей)__ удаление, преобразование и создание столбцов таблицы
Оказывается, что при правильных преобразованиях таблицы можно добиваться лучшего качества прогноза, а также извлекать новую информацию из данных и интерпретировать её для заказчика.

Feature Engineering — это целая методология получения более качественных и более производительных моделей за счёт манипуляций над данными.


## Цели данного модуля:

* Научиться создавать новые признаки в данных с помощью базовых операций со столбцами.
* Освоить методы работы с датой и временем в Pandas.
* Научиться применять собственные функции для преобразования столбцов и создания новых признаков.
* Познакомиться с новым типом данных category и научиться использовать его при работе с данными.

In [59]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


__Старайтесь всегда оставлять переменную с первоначальным DataFrame неизменной, создавайте копию исходной таблицы и совершайте преобразования на ней.__

In [60]:
# создадим копию melb_df с помощью метода copy():
melb_df = melb_data.copy()
melb_df.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


## УДАЛЕНИЕ СТОЛБЦОВ
Основные параметры метода __drop()__
* labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;
* axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;
* inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

Представим, что мы хотим построить модель, которая бы предсказывала цену объекта недвижимости в Мельбурне. Мы можем легко сделать следующие выводы:

* цена объекта никак не зависит от его порядкового номера (столбец index);
* признак, описывающий долготу и широту в виде кортежа Coordinates, дублирует информацию, представленную в столбцах Longitude и Lattitude.

In [61]:
# Удалим столбцы index и Coordinates из таблицы с помощью метода drop(). 
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,2,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


In [62]:
# Альтернативный вариант:

#melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
#melb_df.head()

## МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

 Pandas поддерживает базовые математические операции между столбцами: столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень.

 Причём все операции со столбцами совершаются поэлементно, очень быстро, а самое главное — без написания циклов.
 
 Такая производительность достигается за счёт того, что все математические операции со столбцами выполняются на языке программирования С, что значительно повышает скорость вычислений по сравнению с перебором элементов в цикле. 

In [63]:
# Создадим  total_rooms, в которой будем хранить общее количество комнат в здании. 
# Для этого выполним сложение столбцов с количеством комнат, ванн и спален:

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

In [64]:
# признак MeanRoomsSquare = средней площади одной комнаты для каждого объекта. 
# Для этого разделим площадь здания на полученное ранее общее количество комнат:

melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [65]:
# признак — AreaRatio, коэффициент соотношения площади здания (BuildingArea) 
# и площади участка (Landsize). 
# Для этого разницу двух площадей поделим на их сумму:

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

Что показывает такой коэффициент? AreaRatio лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

* Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.
* Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.
* Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.

Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

In [66]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
#  Для каждой страны рассчитайте плотность населения 
# (количество человек на квадратный километр). 
# Затем по полученным данным рассчитайте среднее по плотностям населения 
# в указанных странах. Ответ округлите до сотых.

countries_df['popul_mean']=countries_df['population']*1000000/countries_df['square']
display(countries_df['popul_mean'].mean())

84.93080566562001

## ФОРМАТ DATETIME
формат для храннения временных артибутов - datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

In [67]:
# можно воспользоваться функцией pandas.to_datetime(). 
# В нашем случае в функции нужно указать параметр dayfirst=True, 
# который будет обозначать, что в первоначальном признаке первым идет день. 
# Преобразуем столбец Date в формат datetime, передав его в эту функцию:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

In [68]:
# Преобразуйте столбец SellerG с наименованиями риелторских компаний 
# в таблице melb_df следующим образом: 
# оставьте в столбце только 49 самых популярных компаний, 
# а остальные обозначьте как 'other'.
# Найдите, во сколько раз минимальная цена объектов недвижимости, 
# проданных компанией 'Nelson', больше минимальной цены объектов, 
# проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.
display(melb_df['SellerG'].nunique())
display(melb_df['SellerG'].value_counts())
popular_sellers =melb_df['SellerG'].value_counts().nlargest(49).index
print(popular_sellers)

melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
display(melb_df['SellerG'])
melb_df[melb_df['SellerG']=='Nelson']['Price'].min()/melb_df[melb_df['SellerG']=='other']['Price'].min()


268

Nelson           1565
Jellis           1316
hockingstuart    1167
Barry            1011
Ray               701
                 ... 
Prowse              1
Luxe                1
Zahn                1
Homes               1
Point               1
Name: SellerG, Length: 268, dtype: int64

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Eview', 'Purplebricks', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'McDonald', 'Burnham',
       'Moonee', 'LITTLE'],
      dtype='object')


0          Biggin
1          Biggin
2          Biggin
3          Biggin
4          Nelson
           ...   
13575       Barry
13576    Williams
13577       Raine
13578     Sweeney
13579     Village
Name: SellerG, Length: 13580, dtype: object

1.297709923664122

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:
* date — дата;
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).


In [69]:
# обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи 
# и понять, за какой интервал времени (в годах) представлены наши данные, 
# а также на какой год приходится наибольшее число продаж:
years_sold = melb_df['Date'].dt.year
print(years_sold)
# В результате обращения к атрибуту datetime melb_df['Date'].dt.year 
# мы получаем объект Series, в котором в качестве значений выступают годы продажи 
# объектов недвижимости. Мы можем занести результат в переменную year_sold 
# и далее работать с ней как с обычным столбцом Series — вычислять максимум,
# минимум и модальное значение.
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [70]:
#Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. 
# Для этого выделим атрибут dt.month и на этот раз занесём результат 
# в столбец MonthSale, а затем найдём относительную частоту продаж 
# для каждого месяца от общего количества продаж — 
# для этого используем метод value_counts() с параметром normalize (вывод в долях):

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

## РАБОТА С ИНТЕРВАЛАМИ

In [71]:
#разницу между датами продаж и заявленной датой, представленной в формате datetime:

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

In [72]:
# тобы превратить количество дней из формата интервала в формат целого числа дней,
# можно воспользоваться аксессором dt для формата timedelta 
# и извлечь из него атрибут days:

display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

In [73]:
# создадим признак возраста объекта недвижимости в годах на момент продажи. 
# Для этого выделим из столбца с датой продажи год и вычтем из него год постройки 
# здания. Результат оформим в виде столбца AgeBuilding:

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])
# так как года кодируются целым числом, в результате мы тоже получаем 
# целочисленный столбец — тип int64 (а не timedelta).

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

In [74]:
# На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, 
# так как, зная год постройки здания, мы автоматически знаем его возраст. 
# Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, 
# так как он является более наглядным, а год постройки удалим из таблицы:

melb_df = melb_df.drop('YearBuilt', axis=1)

In [75]:
# Создайте в таблице melb_df признак WeekdaySale (день недели). 
# Найдите, сколько объектов недвижимости было продано в выходные 
# (суббота и воскресенье), результат занесите в переменную weekend_count. 
# В качестве ответа введите результат вывода переменной weekend_count
melb_df['WeekdaySale']=melb_df['Date'].dt.day_of_week
WdS =melb_df['WeekdaySale'].value_counts()
display(WdS)
WdS.loc[6]+WdS[5]


5    11759
6     1063
0      597
1      133
3       28
Name: WeekdaySale, dtype: int64

12822

Вам представлены данные (в формате csv) об отчётах очевидцев НЛО в США за период с 1930 по 2020 год.

В данных есть следующие признаки:

* "City" — город, где был замечен НЛО;
* "Colors Reported" — цвет объекта;
* "Shape Reported" — форма объекта;
* "State" — обозначение штата;
* "Time" — время, когда был замечен НЛО (данные отсортированы от старых наблюдений к новым). 

Прочитайте данные, сделайте преобразование времени к формату datetime и выполните задания ниже.

In [76]:
data = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv',sep=',')
display(data)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45


In [77]:
data['Date'] = pd.to_datetime(data['Time'], dayfirst=True)
display(data)

,City,Colors Reported,Shape Reported,State,Time,Date
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,1930-01-06 22:00:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,1931-01-06 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,1933-04-18 19:00:00
...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,2000-12-31 23:00:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,2000-12-31 23:00:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,2000-12-31 23:45:00
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,2000-12-31 23:45:00


In [78]:
# В каком году отмечается наибольшее количество случаев наблюдения НЛО в США?
years=data['Date'].dt.year
years.mode()

0    1999
Name: Date, dtype: int64

In [79]:
# Найдите средний интервал времени (в днях) между двумя последовательными случаями
# наблюдения НЛО в штате Невада (NV).
# Чтобы выделить дату из столбца Time, 
# можно воспользоваться атрибутом datetime date.
# Чтобы вычислить разницу между двумя соседними датами в столбце, 
# примените к нему метод diff().
# Чтобы перевести интервал времени в дни, воспользуйтесь атрибутом timedelta days.

dates=data[data['State'] == 'NV']['Date'].dt.date
#display(dates)
dates=dates.sort_values()
#display(dates.diff())
dtdiff =abs(dates.diff().dt.days)
#display(dtdiff)
display(dtdiff.mean())

68.79858657243817

## Расширение функциональности за счёт использования пользовательских функций. 

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

In [80]:
# В наших данных есть столбец с адресами объектов недвижимости. 
# Проблема этого столбца в том, что в нём слишком большое количество 
# уникальных значений: почти на каждый объект недвижимости в таблице 
# приходится свой уникальный адрес. Убедимся в этом, вычислив количество
# уникальных значений в столбце с помощью метода nunique():

print(melb_df['Address'].nunique())

13378


Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет статистической значимости, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.

In [81]:
#Обычно подобные признаки удаляют, однако можно поступить умнее: 
# давайте извлечём из признака адреса характеристику подтипа улицы 
# (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим 
# на структуру адреса, выберем несколько строк столбца Address:

print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [82]:
#Итак, адрес строится следующим образом: 
# сначала указывается номер дома и корпус, 
# после указывается название улицы, 
# а в конце — подтип улицы, 
# но в некоторых случаях к подтипу добавляется географическая отметка 
# (N — север, S — юг и т. д.), она нам не нужна . 
# Для того чтобы выделить подтип улицы, на которой находится объект, 
# можно использовать следующую функцию:

# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.

# в списке подтипов улиц street_types можно заметить подтипы, 
# которые именуются различным образом, но при этом обозначают одинаковые вещи. 
# Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. 
    repl = {'Avenue':'Av', 'Boulevard':'Bvd', 'Parade':'Pde', 'Crescent': 'Cr','Grove':'Gr','Terrace':'Tce'}
    if street_type in repl.keys():
        street_type = repl[street_type]
    return street_type

In [83]:
#Теперь применим эту функцию к столбцу c адресом. 
# Для этого передадим функцию get_street_type в аргумент метода столбца apply().
# В результате получим объект Series, который положим в переменную street_types:

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [84]:
#сколько уникальных значений у нас получилось:

print(street_types.nunique())

50


In [85]:
#У нас есть 56 уникальных значений. Однако наш результат можно улучшить. 
# Давайте посмотрим на частоту каждого подтипа с помощью метода value_counts:

display(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            316
Pde           226
Pl            169
Cr            161
Cl            100
La             67
Bvd            66
Tce            48
Wy             40
Cct            25
Hwy            24
Sq             11
Cir             7
Strand          7
Esplanade       6
Fairway         4
Gdns            4
Grn             4
Mews            4
Righi           3
Crossway        3
Ridge           2
Victoria        2
Esp             2
Crofts          2
Qy              1
Grange          1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Summit          1
Grand           1
Gra             1
Eyrie           1
Glade           1
Dell            1
East            1
Loop            1
Res             1
Athol           1
Cove            1
Nook            1
Corso           1
Name: Address, dtype: int64

In [86]:
# можно увидеть, что есть группа наиболее популярных подтипов улиц, 
# а дальше частота подтипов быстро падает.
# давайте применим очень распространённый метод уменьшения 
# количества уникальных категорий — выделим n подтипов, которые встречаются 
# чаще всего, а остальные обозначим как 'other' (другие).
# Для этого к результату метода value_counts применим метод nlargest(), 
# который возвращает n наибольших значений из Series. 
# Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. 
# Извлечём их названия с помощью атрибута index, 
# а результат занесём в переменную popular_stypes:

popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


In [87]:
# Теперь введём lambda-функцию, которая будет проверять, 
# есть ли строка x в этом перечне, и, если это так, 
# lambda-функция будет возвращать x, в противном случае она будет возвращать 
# строку 'other'. Наконец, применим такую функцию к Series street_types, 
# полученной ранее, а результат определим в новый столбец таблицы StreetType:

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [88]:
# Посмотрим на результирующее число уникальных подтипов:

print(melb_df['StreetType'].nunique())

11


In [89]:
# Теперь, у нас нет потребности хранить признак Address, так как, 
# если конкретное местоположение объекта всё же и влияет на его стоимость, 
# то оно определяется столбцами Longitude и Lattitude. 
# Удалим его из нашей таблицы:

melb_df = melb_df.drop('Address', axis=1)

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:
1. Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.
2. Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной
3. Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.
4. Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.Когда вы выбрали оптимальное число, переходите к шагу 5.
5. Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».

In [90]:
#мы создали признак WeekdaySale в таблице melb_df — день недели продажи.
# Из полученных в задании результатов можно сделать вывод, 
# что объекты недвижимости в Мельбурне продаются преимущественно по выходным 
# (суббота и воскресенье).
#Напишите функцию get_weekend(weekday), 
# оторая принимает на вход элемент столбца WeekdaySale и возвращает 1, 
# если день является выходным, и 0 — в противном случае, 
# и создайте столбец Weekend в таблице melb_df с помощью неё.
#Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, 
# проданного в выходные дни. 

# На вход данной функции поступает строка с адресом.
def get_weekend(weekday):
    if weekday > 4:
        return 1
    return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
display(melb_df['Weekend'])
display(melb_df[melb_df['Weekend']==1]['Price'].mean())

0        1
1        0
2        1
3        1
4        1
        ..
13575    1
13576    1
13577    1
13578    1
13579    1
Name: Weekend, Length: 13580, dtype: int64

1081198.6406956792

In [91]:
# Преобразуйте столбец SellerG с наименованиями риелторских компаний 
# в таблице melb_df следующим образом: 
# оставьте в столбце только 49 самых популярных компаний, 
# а остальные обозначьте как 'other'.
# Найдите, во сколько раз минимальная цена объектов недвижимости, 
# проданных компанией 'Nelson', больше минимальной цены объектов, 
# проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.
display(melb_df['SellerG'].nunique())
display(melb_df['SellerG'].value_counts())
popular_sellers =melb_df['SellerG'].value_counts().nlargest(49).index
print(popular_sellers)

melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
display(melb_df['SellerG'])
melb_df[melb_df['SellerG']=='Nelson']['Price'].min()/melb_df[melb_df['SellerG']=='other']['Price'].min()


50

Nelson           1565
Jellis           1316
other            1199
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Woodards          301
Fletchers         301
Jas               243
Greg              239
McGrath           222
Sweeney           216
Noel              205
Miles             196
RT                184
Gary              170
Harcourts         168
Hodges            157
YPA               154
Stockdale         150
Village           125
Kay               119
Raine             116
Williams          111
Love              109
Douglas            97
Chisholm           77
RW                 70
Rendina            66
HAR                62
O'Brien            61
C21                57
Collins            56
Cayzer             52
Eview              51
Purplebricks       51
Philip             48
Buckingham         46
Bells              44
Thomson            42
Nick               40
Alexkarbon

Index(['Nelson', 'Jellis', 'other', 'hockingstuart', 'Barry', 'Ray',
       'Marshall', 'Buxton', 'Biggin', 'Brad', 'Woodards', 'Fletchers', 'Jas',
       'Greg', 'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary',
       'Harcourts', 'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine',
       'Williams', 'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR',
       'O'Brien', 'C21', 'Collins', 'Cayzer', 'Eview', 'Purplebricks',
       'Philip', 'Buckingham', 'Bells', 'Thomson', 'Nick', 'Alexkarbon',
       'McDonald', 'Burnham', 'Moonee'],
      dtype='object')


0          Biggin
1          Biggin
2          Biggin
3          Biggin
4          Nelson
           ...   
13575       Barry
13576    Williams
13577       Raine
13578     Sweeney
13579     Village
Name: SellerG, Length: 13580, dtype: object

1.297709923664122

In [92]:
def get_experience(arg):
    """
    Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
    Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
    """
    words = list(arg.split())
    years = 0 #Количество лет
    months = 0 #Количество месяцев
    for i,wrd in enumerate(words):
        if wrd in ['год', 'года', 'лет']:
            years = int(words[i-1])
        elif wrd in ['месяц', 'месяцев', 'месяца']:
            months = int(words[i-1])
    return years*12+months

if __name__ == '__main__':
    experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
    experience_month = experience_col.apply(get_experience)
    print(experience_month)

0    99
1    41
2    21
3     3
4    72
dtype: int64


## Тип данных Category
Под числовыми признаками обычно подразумевают признаки, которые отражают количественную меру и могут принимать значения из неограниченного диапазона.

Числовые признаки могут быть:

* дискретными (например, количество комнат, пациентов, дней, отток сотрудников);
* непрерывными (например, масса, цена, площадь).
Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.

Категориальные признаки могут быть:

* номинальными (например, пол, национальность, район);
* порядковыми (например, уровень образования, уровень комфорта, стадия заболевания).
Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных object.
Однако это не всегда так. Например, созданный нами ранее признак месяца продажи кодируется числом (от 1 до 12), но на самом деле является категориальным, поскольку диапазон его значений ограничен и каждому числу мы можем поставить в соответствие название месяца.

Решение, какой признак отнести к классу категорий, остаётся за исследователем.

анализ и предобработка категориальных признаков отличается от предобработки числовых признаков.

In [93]:
#Давайте определим число уникальных категорий в каждом столбце таблицы melb_df. 
# Для этого создадим вспомогательную таблицу unique_counts:

# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,Method,5,object
3,WeekdaySale,5,int64
4,Regionname,8,object
5,Bathroom,9,int64
6,Rooms,9,int64
7,Car,11,int64
8,StreetType,11,object
9,Bedroom,12,int64


Разберём код подробнее:

1. Создаём пустой список, в который будем добавлять кортежи: имя столбца, количество уникальных значений в нём и тип столбца.

2. В цикле перебираем имена столбцов, которые получаем с помощью атрибута columns. В переменной col на каждой итерации находятся имена столбцов — обращаемся к ним в цикле и извлекаем число уникальных элементов с помощью метода nunique(), а также тип столбца с помощью атрибута dtypes. Результат заносим в кортеж и добавляем его в список.

3. Из списка с кортежами (имя столбца, количество уникальных значений в нём, тип столбца) создаём DataFrame, даём названия его столбцам: Column_Name, Num_unique и Type.

4. Сортируем таблицу по столбцу Num_unique в порядке возрастания количества уникальных элементов с помощью метода sort_values() и выводим результат на экран.

можно увидеть резкий скачок количества уникальных значений, начиная с 14 строки таблицы, где число уникальных значений составляет 152
Условимся, что категориальными будем считать признаки, у которых число уникальных категорий меньше 150. 

Однако учтём, что признак Date (дата продажи), преобразованный нами ранее в формат datetime, является временным признаком, поэтому далее не будем его воспринимать как категориальный. 

К тому же в наш потенциальный список попали количественные столбцы Rooms, Car, Bedroom и Bathroom. Договоримся, что мы не будем относить их к разряду категориальных, однако, как упоминалось ранее, такое тоже вполне возможно.

Примечание. Ещё раз подчеркиваем, что такая классификация признаков является исключительно субъективной и специфична для задачи.

## Для хранения и оптимизации работы с категориальными признаками в Pandas предусмотрен специальный тип данных — __category.__
→ Этот тип данных является гибридным: внешне он выглядит как строка, но внутренне представлен массивом целых чисел. Так как данные вместо изначальных строк хранятся в памяти как число, то объём памяти, занимаемой таблицей при использовании типа category, резко уменьшается, что повышает эффективность хранения и работы с таблицей.

→ Более того, этот тип данных расширяет возможности работы с категориальными признаками: мы можем легко преобразовывать категории, строить графики по таким данным (что сложно сделать для типа данных object). Также резко повышается производительность операций, совершаемых с такими столбцами.

In [94]:
# Для начала, выведем информацию о памяти, занимаемой текущей таблицей, с помощью метода info():

display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  object        
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  object        
 5   SellerG          13580 non-null  object        
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  objec

None

In [95]:
# Сделаем преобразование столбцов к типу данных category:

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

Разберём код подробнее:

1. Задаём список столбцов, которые мы не берём в рассмотрение (cols_to_exclude), а также условленный нами ранее порог уникальных значений столбца max_unique_count.

2. В цикле перебираем имена столбцов, и, если число уникальных категорий меньше заданного порога и имён столбцов нет в списке cols_to_exclude, то с помощью метода astype() приводим столбец к типу данных category.

3. Итоговый объём памяти — 1.9 Мб. В результате такого преобразования объём памяти, занимаемый таблицей, уменьшился почти в 1.5 раза. Это впечатляет!

Особенно хорошо такое преобразование работает на действительно больших данных, где число строк превышает сотни тысяч или миллионы. Иногда изменение типов может уменьшить объём памяти в десятки раз и существенно увеличить производительность.

## ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY

У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их. 

In [96]:
#Например, с помощью атрибута этого аксессора categories 
# мы можем получить список уникальных категорий в столбце Regionname:

print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [97]:
#С помощью метода аксессора rename_categories() можно легко переименовать 
# текущие значения категорий. Для этого в данный метод нужно передать словарь, 
# ключи которого — старые имена категорий, а значения — новые.
#Рассмотрим на примере: переименуем категории признака типа постройки 
# Type — заменим их на полные названия 
# (напомним, u — unit, h — house, t — townhouse).

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

## ПОДВОДНЫЕ КАМНИ

In [98]:
# А теперь представим ситуацию, что появилась новая партия домов 
# и теперь мы продаём и квартиры (flat). 
# Создадим объект Series new_houses_types, 
# в котором будем хранить типы зданий новой партии домов. 
# Преобразуем тип new_houses_types в такой же тип, 
# как и у столбца Type в таблице melb_data, и выведем результат на экран:

new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

тип данных category хранит только категории, которые были объявлены при его инициализации. При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.

In [99]:
# Можно добавить категорию flat в столбец Type с помощью метода 
# акссесора cat add_categories(), в который достаточно просто передать 
# имя новой категории:

melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

Из данного примера можно сделать вывод, что если набор категорий в столбце жёстко не зафиксирован и может обновляться в процессе работы, то тип category не является подходящим типом данных для этого столбца или необходимо постоянно писать проверки при обновлении таблицы.

Теперь, когда мы рассмотрели основные моменты и нюансы работы с типом данных category, можно сформулировать несколько рекомендаций по его использованию:

1. Необязательно каждый раз преобразовывать категориальные данные в тип данных category. Зачастую это делается исключительно для оптимизации работы с большими данными.

2. Если набор данных занимает значительный процент используемой оперативной памяти, рассмотрите возможность использования типа category.

3. Если у вас очень серьёзные проблемы с производительностью, обратите внимание на использование типа category.

4. Если вы решили использовать тип category, будьте осторожны при добавлении новой информации в вашу таблицу. Убедитесь, что вы собрали всю необходимую информацию, произведите предобработку данных и только после этого используйте преобразование типов.

In [100]:
# При преобразовании столбцов таблицы о недвижимости к типу category 
# мы оставили без внимания столбец Suburb (пригород). Давайте исправим это.
# С помощью метода info() узнайте, сколько памяти занимает таблица melb_df.
display(melb_df.info())
# Преобразуйте признак Suburb следующим образом: 
# оставьте в столбце только 119 наиболее популярных пригородов, 
# остальные замените на 'other'.
display(melb_df['Suburb'].nunique())
display(melb_df['Suburb'].value_counts())
popular_Suburb =melb_df['Suburb'].value_counts().nlargest(119).index
print(popular_Suburb)

melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_Suburb else 'other')
display(melb_df['Suburb'])
# Приведите данные в столбце Suburb к категориальному типу.
melb_df['Suburb'] = melb_df['Suburb'].astype('category') # преобразуем тип столбца
# В качестве ответа запишите разницу между объёмом занимаемой памяти 
# о преобразования (который мы получили ранее в модуле) и после него в Мб. 
# Ответ округлите до десятых.
display(melb_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

314

Reservoir         359
Richmond          260
Bentleigh East    249
Preston           239
Brunswick         222
                 ... 
Sandhurst           1
Bullengarook        1
Croydon South       1
Montrose            1
Monbulk             1
Name: Suburb, Length: 314, dtype: int64

Index(['Reservoir', 'Richmond', 'Bentleigh East', 'Preston', 'Brunswick',
       'Essendon', 'South Yarra', 'Glen Iris', 'Hawthorn', 'Coburg',
       ...
       'Albion', 'Hoppers Crossing', 'Sunbury', 'Greensborough', 'Bundoora',
       'Hughesdale', 'Chadstone', 'Caulfield North', 'Mont Albert',
       'Alphington'],
      dtype='object', length=119)


0          Abbotsford
1          Abbotsford
2          Abbotsford
3          Abbotsford
4          Abbotsford
             ...     
13575           other
13576    Williamstown
13577    Williamstown
13578    Williamstown
13579      Yarraville
Name: Suburb, Length: 13580, dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

# выгрузка результата

In [101]:
melb_df.to_csv('data/melb_df.csv', sep=',')

## данные о велопоездках клиентов компании Citi Bike (США), специализирующейся на прокате велосипедов.

атасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

* starttime — время начала поездки (дата, время);
* stoptime — время окончания поездки (дата, время);
* start station id — идентификатор стартовой стоянки;
* start station name — название стартовой стоянки;
* start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
* end station id — идентификатор конечной стоянки;
* end station name — название конечной стоянки;
* end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
* bikeid — идентификатор велосипеда;
* usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
* birth year — год рождения клиента;
* gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).

In [102]:
# загрузим
citibike_data = pd.read_csv('data/citibike-tripdata.csv', sep=',')
citibike_data.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


In [103]:
# создадим копию
citibike_df = citibike_data.copy()
citibike_df.tail()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
299995,2018-09-05 19:08:27.8460,2018-09-05 19:15:51.4940,494.0,W 26 St & 8 Ave,40.747348,-73.997236,448.0,W 37 St & 10 Ave,40.756604,-73.997901,19531,Subscriber,1990,2
299996,2018-09-05 19:08:28.3700,2018-09-05 19:20:01.5080,3016.0,Kent Ave & N 7 St,40.720368,-73.961651,3086.0,Graham Ave & Conselyea St,40.715143,-73.944507,34020,Subscriber,1982,1
299997,2018-09-05 19:08:27.5090,2018-09-05 19:13:40.5060,3686.0,Gansevoort St & Hudson St,40.739448,-74.005070,168.0,W 18 St & 6 Ave,40.739713,-73.994564,33220,Subscriber,1991,1
299998,2018-09-05 19:08:29.2300,2018-09-05 20:04:29.3220,254.0,W 11 St & 6 Ave,40.735324,-73.998004,499.0,Broadway & W 60 St,40.769155,-73.981918,34744,Subscriber,1975,0
299999,2018-09-05 19:08:29.7110,2018-09-05 19:12:14.7620,3496.0,1 Ave & E 110 St,40.792327,-73.938300,3502.0,Lexington Ave & E 111 St,40.795412,-73.944123,18292,Subscriber,1966,1


In [104]:
#Сколько пропусков в столбце start station id?
citibike_df.info()
print(300000-299831)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

In [105]:
# Найдите идентификатор самой популярной стартовой стоянки. 
# Запишите идентификатор в виде целого числа.
citibike_df['start station id'].mode()[0]

281.0

In [106]:
# Велосипед с каким идентификатором является самым популярным?
citibike_df['bikeid'].mode()[0]

33887

In [107]:
# Какой тип клиентов (столбец usertype) является преобладающим — 
# Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа 
# среди общего количества клиентов. 
# Ответ округлите до сотых.
citibike_df['usertype'].value_counts(normalize=True)
#mode_usertype = data['usertype'].mode()[0]
#count_mode_user = data[data['usertype'] == mode_usertype].shape[0]
#print(round(count_mode_user / data.shape[0], 2))

Subscriber    0.774007
Customer      0.225993
Name: usertype, dtype: float64

In [108]:
# Кто больше занимается велоспортом — мужчины или женщины? 
# В ответ запишите число поездок для той группы, у которой их больше.
citibike_df['gender'].value_counts().max()

#male_count = data[data['gender'] == 1].shape[0]
#female_count = data[data['gender'] == 0].shape[0]
#print(max([male_count, female_count]))

183582

In [109]:
# Число уникальных стартовых и конечных стоянок, 
# которыми воспользовались клиенты, не равны друг другу
display(citibike_df['start station id'].nunique())
display(citibike_df['end station id'].nunique())
# В рассматриваемые дни минимальный возраст клиента составлял 10 лет
citibike_years= pd.to_datetime(citibike_df['starttime']).dt.year - citibike_df['birth year']
print(citibike_years.min())
# Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, 
# является стоянка с названием "Eastern Pkwy & Washington Ave"
citibike_df['start station name'].value_counts()
# Наибольшее количество поездок завершается на стоянке 
# под названием "Liberty Light Rail"
citibike_df['end station name'].value_counts()



759

765

16


West St & Chambers St          1963
12 Ave & W 40 St               1909
Pershing Square North          1850
Central Park S & 6 Ave         1828
E 17 St & Broadway             1804
                               ... 
Exchange Place                    3
Union St                          1
Warren St                         1
Montrose Ave & Bushwick Ave       1
Liberty Light Rail                1
Name: end station name, Length: 765, dtype: int64

In [110]:
# Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?
citibike_df = citibike_df.drop(['start station id', 'end station id'], axis=1)
citibike_df.shape[1]
#citibike_df.info()

12

In [111]:
# Замените признак birth year на более понятный признак возраста клиента age. 
# Годом отсчёта возраста выберите 2018 год. 
# Столбец birth year удалите из таблицы. 
# Сколько поездок совершено клиентами старше 60 лет?
citibike_df['age'] = 2018 - citibike_df['birth year']
#citibike_df = citibike_df.drop(['birth year'], axis=1)
citibike_df[citibike_df['age']>60].shape[0]


11837

In [112]:
# Создайте признак длительности поездки trip duration. 
# Для этого вычислите интервал времени между временем окончания поездки 
# (stoptime) и её началом (starttime). 
# Сколько целых минут длилась поездка под индексом 3 в таблице?

citibike_df['trip duration'] = pd.to_datetime(citibike_df['stoptime']) - pd.to_datetime(citibike_df['starttime'])
display(citibike_df.loc[3, 'trip duration'])
# нет атрибута для минут но это целые в разнице


Timedelta('0 days 00:07:16.837000')

In [113]:
# Создайте «признак-мигалку» weekend, который равен 1, 
# если поездка начиналась в выходной день (суббота или воскресенье), 
# и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
def get_weekend (arg):
    if arg > 4:
        return 1
    else: return 0

stime = pd.to_datetime(citibike_df['starttime']).dt.dayofweek
citibike_df['weekend'] = stime.apply(lambda x: 1 if ( x > 4) else 0)
#citibike_df['weekend'] = stime.apply(get_weekend)
print(citibike_df['weekend'].value_counts())

#weekday = data['starttime'].dt.dayofweek
#data['weekend'] = weekday.apply(lambda x: 1 if x ==5 or x == 6 else 0)
#data['weekend'].sum()


0    184865
1    115135
Name: weekend, dtype: int64


In [114]:
# Создайте признак времени суток поездки time_of_day. 
# Время суток будем определять из часа начала поездки. Условимся, что:
# поездка совершается ночью (night), 
# если её час приходится на интервал от 0 (вкл) до 6 (вкл) часов;
# поездка совершается утром (morning), 
# если её час приходится на интервал от 6 (не вкл) до 12 (вкл) часов;
# поездка совершается днём (day), 
# если её час приходится на интервал от 12 (не вкл ) до 18 (вкл) часов;
# поездка совершается вечером (evening), 
# если её час приходится на интервал от 18 (не вкл) до 23 часов (вкл).
# Во сколько раз количество поездок, совершённых днём, больше, 
# чем количество поездок, совёршенных ночью, 
# за представленный в данных период времени? Ответ округлите до целых.

def get_interval (arg):
    if arg < 7:
        return 'night'
    elif arg < 13:
        return 'morning'
    elif arg < 19:
        return 'day'
    else: return 'evening'

shour = pd.to_datetime(citibike_df['starttime']).dt.hour
citibike_df['time_of_day'] = shour.apply(get_interval)
print(citibike_df['time_of_day'].value_counts())
#data['time_of_day'] = data['starttime'].dt.hour.apply(get_time_of_day)
a = citibike_df[citibike_df['time_of_day'] == 'day'].shape[0]
b = citibike_df[citibike_df['time_of_day'] == 'night'].shape[0]
print(round(a / b))


day        143012
morning     95530
evening     46373
night       15085
Name: time_of_day, dtype: int64
9
